# Universidade Federal de Minas Gerais
## Computação Evolucionária - TTC
### Trabalho Prático 3

Daniela Amaral Sampaio - 2017074351

Matheus Brito Faria - 2017074386

## 1. Introdução

O trabalho prático 3 tem como objetivo implementar o algoritmo de evolução diferencial tomando como base o pseudocódigo fornecido pelo professor e testar os seguintes problemas multimodais de otimização conínua:

```
a.   peaks, para ­3 <= x1 <= 3, ­3 <= x2 <= 3 com N = 100; mínimo global em
x*=[0.228,­1.625] com f(x*) = ­6.5511;

b.   rastrigin, para ­2 <= x1 <= 2, ­2<= x2 <= 2 com N = 100; mínimo global em
x*=[0,0] com f(x*) = ­20.
```

A Evolução Diferencial (ED), que constitui um dos mais recentes métodos de otimização evolucionária, é um algoritmo eficiente que,geralmente apresenta rápida convergência na busca das soluções em problemas de otimização contínua. A ED se baseia nos mecanismos de seleção natural e na genética de populações, e utiliza operadores de mutação, cruzamento e seleção para gerar novos indivíduos em busca do mais adaptado e se destaca pela pequena quantidade de parâmetros utilizados. 

In [ ]:
import numpy as np
import copy

## 2. Implementação
Para realizar a implementação foi implementada a função ```def run_differential_evolution
``` 
e foram utilizadas as duas funções objetivos, *peaks* e *rastrigin*, fornecidas pelo professor, para realizar os testes do código.

### 2.1. Função Objetivo *peaks*

In [ ]:
def peaks(x):
    x = x.T
    F = (
        3 * (1 - x[0]) ** 2 * np.exp(-(x[0] ** 2) - (x[1] + 1) ** 2)
        - 10 * (x[0] / 5 - x[0] ** 3 - x[1] ** 5) * np.exp(-x[0] ** 2 - x[1] ** 2)
        - 1 / 3 * np.exp(-((x[0] + 1) ** 2) - x[1] ** 2)
    )
    return F

### 2.2. Função Objetivo *rastrigin*

In [ ]:
def rastrigin(x):
    x = x.reshape(1, -1).T
    Q = np.eye(len(x))
    X = Q.dot(x)

    n = len(X)
    F = 0
    
    for i in range(n):
        F = F + X[i]**2 - 10*np.cos(2*np.pi*X[i])
    
    return F[0]

### 2.3. Função Evolução Diferencial

Inicia-se criando as entradas necessárias, como o tamanho da população, dimensão do problema, probabilidade de recombinação, etc. Após isso, inicia-se a população e começa a realizar a mutação. Após isso, os indivíduos da população corrente são recombinados com os indivíduos da população mutante. Logo após, é realizado a performance de cada população e feita a seleção dos melhores indivíduos, printando na tela esses valores referentes a cada iteração. O processo acaba quando se finalizam o máximo de iterações indicados.

In [ ]:
def run_differential_evolution(
    population_size: int,
    number_variables: int,
    lower_bound: int,
    upper_bound: int,
    probability_recombination: float,
    scale_factor: float,
    objective_function,
    max_iterations: int,
):

    random_generator = np.random.default_rng()

    population = random_generator.uniform(
        low=lower_bound, high=upper_bound, size=(population_size, number_variables)
    )

    iteration = 0
    while iteration <= max_iterations:
        mutate_population = copy.deepcopy(population)

        for individual_index in range(population_size):
            recombinations_indexes = random_generator.choice(
                population_size, size=3, replace=True
            )
            should_mutate = random_generator.random() <= probability_recombination

            if should_mutate:
                variable_to_mutate = random_generator.choice(number_variables)

                mutate_population[individual_index][variable_to_mutate] = population[
                    recombinations_indexes[0]
                ][variable_to_mutate] + scale_factor * (
                    population[recombinations_indexes[1]][variable_to_mutate]
                    - population[recombinations_indexes[2]][variable_to_mutate]
                )

        performance_population = np.array(
            [objective_function(individual) for individual in population]
        )

        performance_mutation = np.array(
            [objective_function(individual) for individual in mutate_population]
        )

        best_population = np.min(performance_population)

        best_mutation = np.min(performance_mutation)

        if iteration % 10 == 0:

            if best_population < best_mutation:
                print(
                    f"Iteration {iteration} - f(x*)={best_population} where x*="
                    f"{population[np.where(performance_population==best_population)][0]}"
                )
            else:
                print(
                    f"Iteration {iteration} - f(x*)={best_mutation} where x*="
                    f"{population[np.where(performance_mutation==best_mutation)][0]}"
                )

        index_selection = performance_mutation < performance_population

        population[index_selection] = mutate_population[index_selection]

        iteration += 1


## 3. Resultados

Nesse tópico, faremos o teste do algoritmo implementado para cada uma das duas funções objetivos fornecidas pelo professor e analisaremos se os resultados foram satisfatórios.

### 3.1. Resultados da Função Objetivo *peaks*

Inicializa-se com uma população de tamanho 100, com limite inferior igual a -3 e limite superior igual a 3, probabilidade de recombinação de 0.8, fator de escala igual a 0.7, utilizando a função objetivo de *peaks* e com um máximo de iterações igual a 200.

In [ ]:
run_differential_evolution(
    population_size=100,
    number_variables=2,
    lower_bound=-3,
    upper_bound=3,
    probability_recombination=0.8,
    scale_factor=0.7,
    objective_function=peaks,
    max_iterations=200,
)

Iteration 0 - f(x*)=-5.9425378881400235 where x*=[ 0.33208068 -1.81422557]
Iteration 10 - f(x*)=-6.5104317921345025 where x*=[ 0.16376418 -1.62337747]
Iteration 20 - f(x*)=-6.541563975484321 where x*=[ 0.21088974 -1.64948385]
Iteration 30 - f(x*)=-6.541563975484321 where x*=[ 0.21088974 -1.64948385]
Iteration 40 - f(x*)=-6.550370887014712 where x*=[ 0.22024347 -1.62990041]
Iteration 50 - f(x*)=-6.55048371094662 where x*=[ 0.22780168 -1.6323005 ]
Iteration 60 - f(x*)=-6.55106839647637 where x*=[ 0.22755591 -1.62359136]
Iteration 70 - f(x*)=-6.55106839647637 where x*=[ 0.22755591 -1.62359136]
Iteration 80 - f(x*)=-6.55106839647637 where x*=[ 0.22755591 -1.62359136]
Iteration 90 - f(x*)=-6.55106839647637 where x*=[ 0.22755591 -1.62359136]
Iteration 100 - f(x*)=-6.55106839647637 where x*=[ 0.22755591 -1.62359136]
Iteration 110 - f(x*)=-6.551094369174478 where x*=[ 0.22664053 -1.62477162]
Iteration 120 - f(x*)=-6.551094369174478 where x*=[ 0.22664053 -1.62477162]
Iteration 130 - f(x*)=-6.55

Visto que o objetivo seria o mínimo global em x*=[0.228 , -1.625] com f(x*) = 6.5511, o teste o algoritmo atendeu com sucesso, visto que o resultado na iteração 200 foi de x*=[ 0.2281035 , -1.62556836] e f(x*)=-6.551133316797472.

### 3.2. Resultados da Função Objetivo *rastrigin*

Inicializa-se com uma população de tamanho 100, com limite inferior igual a -2 e limite superior igual a 2, probabilidade de recombinação de 0.6, fator de escala igual a 0.7, utilizando a função objetivo de *rastrigin* e com um máximo de iterações igual a 200.

In [ ]:
run_differential_evolution(
    population_size=100,
    number_variables=2,
    lower_bound=-2,
    upper_bound=2,
    probability_recombination=0.6,
    scale_factor=0.7,
    objective_function=rastrigin,
    max_iterations=200,
)

Iteration 0 - f(x*)=-18.67579927128467 where x*=[-1.02204254 -0.03049022]
Iteration 10 - f(x*)=-19.740412872454687 where x*=[ 0.0195475  -0.03049022]
Iteration 20 - f(x*)=-19.740412872454687 where x*=[ 0.0195475  -0.03049022]
Iteration 30 - f(x*)=-19.998967740579772 where x*=[-0.00090437 -0.116825  ]
Iteration 40 - f(x*)=-19.998967740579772 where x*=[-0.00090437  0.00209411]
Iteration 50 - f(x*)=-19.999783653883224 where x*=[0.00032728 0.00099166]
Iteration 60 - f(x*)=-19.999996161564464 where x*=[-7.14083013e-06  1.38912687e-04]
Iteration 70 - f(x*)=-19.999998852921816 where x*=[-7.14083013e-06 -7.57025977e-05]
Iteration 80 - f(x*)=-19.999999795536112 where x*=[-3.19886979e-05  2.70707708e-06]
Iteration 90 - f(x*)=-19.999999971665645 where x*=[-6.85834503e-06  9.78688390e-06]
Iteration 100 - f(x*)=-19.999999995418232 where x*=[-3.74466644e-06  3.01197353e-06]
Iteration 110 - f(x*)=-19.9999999997737 where x*=[9.49734865e-07 4.88567021e-07]
Iteration 120 - f(x*)=-19.999999999991005 wher

Visto que o objetivo seria o mínimo global em x*=[0 , 0] com f(x*) = 20, o teste o algoritmo atendeu com sucesso, visto que o resultado na iteração 200 foi de x*=[-3.51436616e-09 , 6.65982088e-10] e f(x*)=-20.0.

## 4. Conclusão

Durante a execução do trabalho foi possível utilizar na prática conceitos aprendidos em sala de aula e acredita-se que o resultado final do trabalho tenha sido satisfatório, visto que nos dois testes de *peaks* e *rastrigin* foram obtidos resultados muito próximos ou iguais aos esperados de mínimos global.